### Riddler Classic: 

Today marks the beginning of the Summer Olympics! One of the brand-new events this year is sport climbing, in which competitors try their hands (and feet) at lead climbing, speed climbing and bouldering.

Suppose the event’s organizers accidentally forgot to place all the climbing holds on and had to do it last-minute for their 10-meter wall (the regulation height for the purposes of this riddle). Climbers won’t have any trouble moving horizontally along the wall. However, climbers can’t move between holds that are more than 1 meter apart vertically.

In a rush, the organizers place climbing holds randomly until there are no vertical gaps between climbing holds (including the bottom and top of the wall). Once they are done placing the holds, how many will there be on average (not including the bottom and top of the wall)?

### Solution Thoughts

- We don't need to care about horizontal here
- General process could be to lean on Monte Carlo to do the following:
    - Generate a random float between 0-10
    - Add to array
    - Sort array
    - Check for continuous movement:
        - Move left to right -> is there a gap that exceeds 1? use np.diff for speed
            - If so, repeat
            - If not, there is a clean path
            
### np.diff:

This is vectorized, so much faster than doing something iteration over a list. Documentation: https://numpy.org/doc/stable/reference/generated/numpy.diff.html

Example: 

```python
>>> arr = np.asarray([1,2,3,4,5])
>>> np.diff(arr)
array([1, 1, 1, 1]) # this is what we see
```

Pretty slick, by default it calculates diff of `element i` and `element i + 1`

In [1]:
import numpy as np

In [2]:
# initialize sequence
hold_sequence = np.asarray([])

while True:
    
    # Add new hold
    hold_sequence = np.append(hold_sequence, np.random.uniform(0,10))

    # Sort array
    hold_sequence = np.sort(hold_sequence)

    # check if first element <= 1; need to be close enough to start of 0
    if hold_sequence[0] > 1:
        continue

    # check if last element >= 9; need to be close to end of 10 (within 1)
    if hold_sequence[-1] < 9:
        continue

    # check if there are any gaps > 1, indicating a need to go again
    if np.max(np.diff(hold_sequence)) > 1:
        continue
    
    # Otherwise we are done
    print(f"Total placements: {len(hold_sequence)}")
    break

Total placements: 38


### Run Sim:

I feel dirty always leaning on simulation but alas, it gets us towards an approximate solution. 

In [3]:
from collections import defaultdict

result_dict = defaultdict(int)  # start with 0
sim_list = [100, 1_000, 10_000, 100_000, 1_000_000]

for sim in sim_list:
    for _ in range(sim):

        # initialize sequence
        hold_sequence = np.asarray([])

        while True:

            # Add new hold
            hold_sequence = np.append(hold_sequence, np.random.uniform(0,10))

            # Sort array
            hold_sequence = np.sort(hold_sequence)

            # check if first element <= 1; need to be close enough to start of 0
            if hold_sequence[0] > 1:
                continue

            # check if last element >= 9; need to be close to end of 10 (within 1)
            if hold_sequence[-1] < 9:
                continue

            # check if there are any gaps > 1, indicating a need to go again
            if np.max(np.diff(hold_sequence)) > 1:
                continue

            # Otherwise we are done & can increment list properly
            result_dict[sim] += len(hold_sequence)
            break
    
    # print stats
    print(f"{sim} sims had an average of {result_dict[sim] / sim:.3f} holds")

100 sims had an average of 44.570 holds
1000 sims had an average of 42.354 holds
10000 sims had an average of 43.272 holds
100000 sims had an average of 42.993 holds
1000000 sims had an average of 43.058 holds
